# Dictionaries

Dictionaries are an essential data structure for data storage in python and a great format for communicating with the outside world:

+ configuration files
+ structured data files
+ JSON from web services.

## Dictionaries as flexible data storage

Let's first think in which ways we can normally get structured data from the outside world.

### CSV document

```{}
country,state,locality,collectors, scientific_name
República Dominicana,Santiago,"Loma La Pelona, Coordillera Central","Juan Pérez, Pancho Luis Díaz Ramírez",Pinus occidentalis
```

### Basic JSON

```{json}
{
    "country": "República Dominicana",
    "state": "Santiago",
    "locality": "Loma La Pelona, Coordillera Central",
    "collectors":"Juan Pérez, Pancho Luis Díaz Ramírez",
    "scientific_name": "Pinus occidentalis"
 }
```

### Better JSON

```{json}
{
    "country": "República Dominicana",
    "state": "Santiago",
    "locality": "Loma La Pelona, Coordillera Central",
    "collectors": ["Juan Pérez", "Pancho Díaz"],
    "scientific_name": "Pinus occidentalis"
}
```

### Maybe best JSON

```    
{
    "country": "República Dominicana",
    "state": "Santiago",
    "locality": "Loma La Pelona, Coordillera Central",
    "collectors": [
        {
            "first_name": "Juan",
            "last_name": "Pérez"
        },
        {
            "first_name": "Pancho",
            "last_name": "Díaz Ramírez",
            "middle_name": 'Luis'
        }
    ],
    "taxonomy": {
        "genus":"Pinus",
        "specific_epithet": "occidentalis"
    }
}
```

In [ ]:
record = {
    "country": "República Dominicana",
    "state": "Santiago",
    "locality": "Loma La Pelona, Coordillera Central",
    "collectors": [
        {
            "first_name": "Juan",
            "last_name": "Pérez"
        },
        {
            "first_name": "Pancho",
            "last_name": "Díaz Ramírez",
            "middle_name": 'Luis'
        }
    ],
    "taxonomy": {
        "genus":"Pinus",
        "specific_epithet": "occidentalis"
    }
}

In [ ]:
record["locality"]

In [ ]:
type(record['country'])

In [ ]:
type(record['collectors'])

In [ ]:
record['collectors'][0]['last_name']

### Live examples: Working with dictionaries

#### Create dictionary from scrach

In [ ]:
a = {}

#### Key value pair query syntax

In [ ]:
a['fruits'] = ['apple', 'pinaple', 'banana', 'orange']

#### Dictionary methods

In [ ]:
dir(a)

In [ ]:
record.keys()

In [ ]:
record['collectors'][0].get('middle_name','No data')

## API example

In [ ]:
import requests
import json
import html
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

## idigbio api

In [ ]:
def search_idigbio(params):
    idigbio_base_url = "https://search.idigbio.org/v2/search/records"
    payload = {
        "rq": json.dumps(params)
    }
    response = requests.get(idigbio_base_url, params=payload)
    return response

In [ ]:
response = search_idigbio({"genus": "Asclepias", "country":"United States"})

In [ ]:
len(response.json()['items'])

In [ ]:
response.url

In [ ]:
records = response.json()

In [ ]:
records.keys()

In [ ]:
records['items'][0]['data'].keys()

In [ ]:
records['items'][0].keys()

In [ ]:
records['items'][0]['data'].keys()

##### Dictionary loop key value pairs

In [ ]:
for key, value in records['items'][0]['data'].items():
    print(key)
    print(value)
    print("+=================================+")

In [ ]:
for key, value in records['items'][0]['data'].items():
    print(key, type(value))

Retrieve only numeric fields

In [ ]:
def is_float(value):
    try:
        res = float(value)
        return res        
    except ValueError:
        return False        

In [ ]:
numeric_fields = []
for key, value in records['items'][0]['data'].items():
    if is_float(value):
        numeric_fields.append(key)

In [ ]:
numeric_fields

In [ ]:
# Returns error: example
numeric_records = []
for record in records['items']:
    new_record = {
        "uuid": record['uuid']
    }
    for field in numeric_fields:
        new_record[field] = record['data'][field]
    numeric_records.append(new_record)

In [ ]:
# Better code example using the get() method
numeric_records = []
for record in records['items']:
    new_record = {
        "uuid": record['uuid']
    }
    for field in numeric_fields:
        new_record[field] = record['data'].get(field, None)
    numeric_records.append(new_record)

In [ ]:
numeric_records[0:2]

In [ ]:
years = [int(record['dwc:year']) for record in numeric_records if record.get('dwc:year', 0)]

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.distplot(years, rug=True, kde=False, bins=20);
plt.show;

In [ ]:
latitudes = [float(record['dwc:decimalLatitude']) for record in numeric_records if record.get('dwc:decimalLatitude', 0)]

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.distplot(latitudes, rug=True, kde=False, bins=20);
plt.show;

Species summary

In [ ]:
species_summary = dict()

for record in res['items']:
    
    taxon = record['data']['dwc:scientificName']
    state = record['data'].get('dwc:stateProvince', 'Unknown')
    
    if species_summary.get(taxon,0):
        
        #species_summary[taxon]['states'] = species_summary[taxon]['states'].add(state) 
        species_summary[taxon]['states'].add(state) 
        species_summary[taxon]['count'] += 1
    else:
        species_summary[taxon] = {
            'states': {state,},
            'count': 1
        }

In [ ]:
species_summary

In [ ]:
species_summary.keys()

In [ ]:
len(species_summary.keys())

## Dictionaries as indexes: examples

### Sequence reverse complement

In [ ]:
seq = "TCGGGCCCAAATCTCCGGAG"

In [ ]:
complement = {
    'A': 'T',
    'C': 'G',
    'G': 'C',
    'T': 'A'
}

In [ ]:
# Short version
reverse_complement = "".join(complement.get(base, 'N') for base in reversed(seq))

In [ ]:
# Long version
rc_raw = []
for base in reversed(seq):
    rc_raw.append(complement.get(base,'N'))
reverse_complement = "".join(rc_raw)    

### Compare sequences between two fasta files

In [ ]:
# Parse fasta file so that headers are the keys and the sequences are the values
def parse_fasta(file):
    with open(file) as f:
        sequences = {}
        for line in f:
            if line.startswith('>'):
                new_key = line.strip().replace('>','')
            else:
                sequences[new_key] = sequences.get(new_key, '') + line.strip()
        return sequences

In [ ]:
full_set = parse_fasta('cactacae_gene_features_25feb2020.fasta')
unwanted = parse_fasta('unwanted.fasta')

In [ ]:
keys_full_set = set(full_set.keys())
keys_unwanted = set(unwanted.keys())

In [ ]:
# keys that are in the full set but not in the unwanted file
to_keep = keys_full_set.difference(keys_unwanted)

In [ ]:
len(to_keep)

In [ ]:
with open('good_sequences.fasta', 'w') as out:
    for seq_key in to_keep:
        out.write(f'>{seq_key}\n')
        out.write(f'{full_set[seq_key]}\n')
    

In [ ]:
! grep ">" good_sequences.fasta|wc -l

In [ ]:
! head good_sequences.fasta

#### Create unwanted file for previous section

In [ ]:
sequences = parse_fasta('cactacae_gene_features_25feb2020.fasta')

In [ ]:
# Check number of sequences in file
n_seqs = len(sequences.keys())
print(n_seqs)

In [ ]:
# Count number of sequences using grep in the shell
!grep '>' cactacae_gene_features_25feb2020.fasta| wc -l

In [ ]:
import random
takeout = random.sample(sequences.keys(), k=int(n_seqs*0.2))

In [ ]:
with open('unwanted.fasta', 'w') as out:
    for seq_key in takeout:
        out.write(f'>{seq_key}\n')
        out.write(f'{sequences[seq_key]}\n')

In [ ]:
!head unwanted.fasta

### Count dinucleotides in sequence

In [ ]:
sources = ['https://pythonforbiologists.com/dictionaries']

In [ ]:
good_seqs = parse_fasta('good_sequences.fasta')

In [ ]:
dna = good_seqs['lcl|KY490880.1_gene_1 [gene=PhyC] [location=<1..968] [gbkey=Gene]']


In [ ]:
def count_dinucleotides(dna):
    all_counts = {} 
    bases = ['A','T','G','C'] 
    for base1 in bases: 
        for base2 in bases: 
            dinucleotide = base1 + base2 
            count = dna.count(dinucleotide) 
            if count > 0: 
                all_counts[dinucleotide] = count
    return all_counts

In [ ]:
all_counts = count_dinucleotides(dna)
print(all_counts)

### DNA translation

In [ ]:
sources = [
    'https://pythonforbiologists.com/dictionaries',
    'https://towardsdatascience.com/starting-off-in-bioinformatics-turning-dna-sequences-into-protein-sequences-c771dc20b89f'
]


In [ ]:
GENCODE = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
    'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W'}

In [ ]:
def translate_dna(dna, frame):
    last_codon_start = len(dna) - 2
    protein = ""
    for start in range(frame,last_codon_start,3):
        codon = dna[start:start+3]
        aa = GENCODE.get(codon.upper(), 'X')
        protein = protein + aa
    return protein 

In [ ]:
dna = good_seqs['lcl|KY490880.1_gene_1 [gene=PhyC] [location=<1..968] [gbkey=Gene]']

In [ ]:
translate_dna(dna,2)

In [ ]:
a = "CCTAAGTTT"
last = len(a) - 2
print(last)
for start in range(0, last, 3):
    print(start, start+3)

### Configure sequences as db

In [ ]:
seq_db = []
for seq_key, seq in good_seqs.items():
    record = {
        "seq_id": seq_key,
        "seq": seq,
        "dinucleotide_freq": count_dinucleotides(seq)
    }
    for frame in range(3):
        record[f'translation_frame{frame}'] = translate_dna(seq,frame)
    seq_db.append(record)

In [ ]:
seq_db[0]

# Extra: Group identical sequences: Not working, cannot handle large files

In [ ]:
good_seqs = parse_fasta('good_sequences.fasta')

In [ ]:
import itertools

In [ ]:
key_pairs = itertools.combinations(good_seqs.keys(), r=2)

In [ ]:
seq_db = []
old_field = None
once_off = False
for key_pair in key_pairs:
    if old_field != key_pair[0]:
        if once_off:
            seq_db.append(record)
        once_off = True                
        record = {
            "seq_id": i[0],
            "seq": good_seqs[i[0]],
            "identical_records": []
        }
    if good_seqs[i[0]] == good_seqs[i[1]]:
        record['identical_records'].append(i[1])
    